In [1]:
from plotnine import *
import pandas as pd
import json

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
beadaya_qa = json.load(open("/content/drive/MyDrive/AUB Research/Arabic Dataset/Beadaya/beadaya_qa_raw.json", "r"))
beadaya_columnar=[]
for grade_subject in beadaya_qa.keys():
  grade, subject = grade_subject.replace("10-11-12", "secondary").split("-")
  for qa in beadaya_qa[grade_subject]:

    if "question" not in qa: # some questions are empty
      continue

    columnar = {}
    columnar["grade"] = grade
    columnar["subject"] = subject

    columnar["question_img"] = qa["question"]["img"]
    columnar["question_text"] = qa["question"]["text"]
    columnar["question_math"] = qa["question"]["math"]

    columnar["sub_question_img"] = qa["sub_question"]["img"] if "sub_question" in qa else ""
    columnar["sub_question_text"] = qa["sub_question"]["text"] if "sub_question" in qa else ""
    columnar["sub_question_math"] = qa["sub_question"]["math"] if "sub_question" in qa else ""

    columnar["option_0_img"] = qa["answer"]["option_0"]["img"] if "option_0" in qa["answer"] else ""
    columnar["option_0_text"] = qa["answer"]["option_0"]["text"] if "option_0" in qa["answer"] else ""
    columnar["option_0_math"] = qa["answer"]["option_0"]["math"] if "option_0" in qa["answer"] else ""

    columnar["option_1_img"] = qa["answer"]["option_1"]["img"] if "option_1" in qa["answer"] else ""
    columnar["option_1_text"] = qa["answer"]["option_1"]["text"] if "option_1" in qa["answer"] else ""
    columnar["option_1_math"] = qa["answer"]["option_1"]["math"] if "option_1" in qa["answer"] else ""

    columnar["option_2_img"] = qa["answer"]["option_2"]["img"] if "option_2" in qa["answer"] else ""
    columnar["option_2_text"] = qa["answer"]["option_2"]["text"] if "option_2" in qa["answer"] else ""
    columnar["option_2_math"] = qa["answer"]["option_2"]["math"] if "option_2" in qa["answer"] else ""

    columnar["option_3_img"] = qa["answer"]["option_3"]["img"] if "option_3" in qa["answer"] else ""
    columnar["option_3_text"] = qa["answer"]["option_3"]["text"] if "option_3" in qa["answer"] else ""
    columnar["option_3_math"] = qa["answer"]["option_3"]["math"]   if "option_3" in qa["answer"] else ""

    columnar["option_4_img"] = qa["answer"]["option_4"]["img"] if "option_4" in qa["answer"] else ""
    columnar["option_4_text"] = qa["answer"]["option_4"]["text"] if "option_4" in qa["answer"] else ""
    columnar["option_4_math"] = qa["answer"]["option_4"]["math"] if "option_4" in qa["answer"] else ""

    columnar["correct_option"] = qa["answer"]["correct_option"]
    columnar["link"] = qa["link"]

    beadaya_columnar.append(columnar)

In [ ]:
import pandas as pd
beadaya_df = pd.DataFrame(beadaya_columnar)

# Remove questions with images
beadaya_df = beadaya_df[beadaya_df["question_img"]==""]
beadaya_df = beadaya_df[beadaya_df["sub_question_img"]==""]
beadaya_df = beadaya_df[beadaya_df["option_0_img"]==""]
beadaya_df = beadaya_df[beadaya_df["option_1_img"]==""]
beadaya_df = beadaya_df[beadaya_df["option_2_img"]==""]
beadaya_df = beadaya_df[beadaya_df["option_3_img"]==""]
beadaya_df = beadaya_df[beadaya_df["option_4_img"]==""]
beadaya_df = beadaya_df.drop(["question_img", "sub_question_img", "option_0_img", "option_1_img", "option_2_img", "option_3_img", "option_4_img"], axis=1)

# Remove questions with math jax
beadaya_df = beadaya_df[beadaya_df["question_math"]==""]
beadaya_df = beadaya_df[beadaya_df["sub_question_math"]==""]
beadaya_df = beadaya_df[beadaya_df["option_0_math"]==""]
beadaya_df = beadaya_df[beadaya_df["option_1_math"]==""]
beadaya_df = beadaya_df[beadaya_df["option_2_math"]==""]
beadaya_df = beadaya_df[beadaya_df["option_3_math"]==""]
beadaya_df = beadaya_df[beadaya_df["option_4_math"]==""]
beadaya_df = beadaya_df.drop(["question_math", "sub_question_math", "option_0_math", "option_1_math", "option_2_math", "option_3_math", "option_4_math"], axis=1)


# Remove questions with no correct answer
beadaya_df = beadaya_df[beadaya_df["correct_option"]!=-1]

# Remove questions with ordering question (The student should reorder boxes as an answer)
beadaya_df = beadaya_df[~beadaya_df["question_text"].str.contains("رتب ")]
beadaya_df = beadaya_df[~beadaya_df["question_text"].str.contains("رتبي ")]

# Combine questions with sub question
beadaya_df['question_text'] = beadaya_df.apply(lambda row: row['question_text'] + " " + row['sub_question_text'], axis=1)
beadaya_df = beadaya_df.drop(["sub_question_text"], axis=1)

# Remove those with empty answer
beadaya_df = beadaya_df[beadaya_df["question_text"]!=""]
beadaya_df = beadaya_df[beadaya_df["question_text"]!=" "]

# Remove those with 0, 1, and 5 options
subset = beadaya_df[["option_0_text", "option_1_text", "option_2_text", "option_3_text", "option_4_text"]].copy()
subset['n_options'] = subset.apply(lambda row: sum(value != '' for value in row), axis=1)
erronouns_options = subset['n_options'].apply(lambda v: v in [0, 1, 5])
beadaya_df = beadaya_df.loc[~erronouns_options, :]

beadaya_df

In [ ]:
# questions = []
# for i, row in beadaya_df.iterrows():
#   question = {}
#   question["question"] = row["question_text"].strip()
#   question["answer"] = {}
#   for i in [0, 1, 2, 3]:
#     question["answer"][f"option_{i}"] = row[f"option_{i}_text"]
#   question["answer"][f"correct_option"] = row["correct_option"]
#   question["grade"] = row["grade"]
#   question["subject"] = row["subject"]
#   question["resource"] = row["link"]
#   questions.append(question)
# json.dump(questions, open("/content/drive/MyDrive/AUB Research/Arabic Dataset/Beadaya/beadaya_qa.json", "w"), ensure_ascii=False)

In [ ]:
from plotnine import *

df = beadaya_df[["option_0_text", "option_1_text", "option_2_text", "option_3_text", "option_4_text"]]
df['n_options'] = df.apply(lambda row: sum(value != '' for value in row), axis=1)
df = df.groupby("n_options").agg(count=("n_options", "count")).reset_index()
df

(
    ggplot(df, aes(x='n_options', y='count')) +
    geom_bar(stat='identity', fill='skyblue') +
    geom_text(aes(label='count'), va='bottom', size=10) +
    labs(x='Number of Options', y='Count', title='Bar Chart with Count Labels') +
    theme_minimal()
)

In [ ]:
# Aggregate data
df = beadaya_df.groupby(['grade', 'subject']).agg(count=("grade", "count")).reset_index()

# Plotting the stacked bar chart
(ggplot(df, aes(x='grade', y='count', fill='subject', label='count')) +
 geom_bar(stat='identity') +
 labs(title='Number of Questions Per Grade and Subject', x="Grades", y="Count", fill="Subject") +
 geom_text(position=position_stack(vjust=0.5), format_string='{}', size=10)+
 theme_minimal()
)

In [ ]:
df = beadaya_df.assign(question_token_count = beadaya_df["question_text"].str.strip().str.split(" ").apply(len))
df = df[["question_token_count"]]

max_count = df['question_token_count'].max()
bin_width = 1
(
    ggplot(df, aes(x="question_token_count"))+
    geom_histogram(binwidth=bin_width, fill='blue', color='black') +
    scale_x_continuous(breaks= range(0, max_count + 5, 5)) +
    labs(title='Distribution of Token Counts', x='Token Count', y='Frequency') +
    theme_minimal()
)

In [ ]:
# Token count 1
from IPython.display import HTML
df = beadaya_df.assign(question_token_count = beadaya_df["question_text"].str.strip().str.split(" ").apply(len))
samples = df[df["question_token_count"]==25].sample(n=5)
result = ""
for i, (index, row) in enumerate(samples.iterrows()):
  result+= f"{i+1}."+ f" {row['question_text']}" + ": <br/>"
  for j,label in enumerate(["أ", "ب", "ج", "د"]):
    column = f"option_{j}_text"
    if row[column]!="":
      result+= f"({label})." + f" {row[column]}" + " <br/>"
  result+= "<br/>"
HTML(f"<div dir='rtl' style='background-color: black; color: white; padding: 20px; font-size: 14px;'>{result}</div>")